In [143]:
import numpy as np
import pandas as pd
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix
#from sklearn.cluster import KMeans

In [144]:
df = pd.read_csv('C:/Users/Personal/Desktop/MDLP1/CPA project/final_df.csv')
df.head()

,age,Income,Education,Marital_Status,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Response,tot_children
0,Adult,58138.0,Graduation,Single,58,635,88,546,172,88,88,3,8,10,4,7,1,0
1,Senior_Citizen,46344.0,Graduation,Single,38,11,1,6,2,1,6,2,1,1,2,5,0,2
2,Adult,71613.0,Graduation,Married,26,426,49,127,111,21,42,1,8,2,10,4,0,0
3,Adult,26646.0,Graduation,Married,26,11,4,20,10,3,5,2,2,0,4,6,0,1
4,Adult,58293.0,Higher education,Married,94,173,43,118,46,27,15,5,5,3,6,5,0,1


In [145]:
from sklearn.preprocessing import LabelEncoder
#dsdgd
df1=df.copy()
lblEn = LabelEncoder()
df1['Education'] =lblEn.fit_transform(df1['Education'])
df1['Marital_Status'] =lblEn.fit_transform(df1['Marital_Status'])
df1['age'] =lblEn.fit_transform(df1['age'])

In [146]:
#df1=df1.drop(['age'],axis=1)
df1.head()

,age,Income,Education,Marital_Status,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Response,tot_children
0,0,58138.0,0,2,58,635,88,546,172,88,88,3,8,10,4,7,1,0
1,1,46344.0,0,2,38,11,1,6,2,1,6,2,1,1,2,5,0,2
2,0,71613.0,0,0,26,426,49,127,111,21,42,1,8,2,10,4,0,0
3,0,26646.0,0,0,26,11,4,20,10,3,5,2,2,0,4,6,0,1
4,0,58293.0,1,0,94,173,43,118,46,27,15,5,5,3,6,5,0,1


In [159]:
scaler = StandardScaler()
scaler.fit(df1.drop('Response',axis=1))

StandardScaler()

In [165]:
X = scaler.transform(df1.drop('Response',axis=1))

In [166]:
#Random Over Sampling for balancing Response data
from imblearn import over_sampling
from imblearn.over_sampling import RandomOverSampler

# Separate input features and target
y = df1.Response
#X = df1.drop('Response', axis=1)

# setting up testing and training sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

rov = RandomOverSampler()
X, y= rov.fit_resample(X,y)

In [167]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.75,random_state=20)

###### Selection of Best Model

In [168]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit

def find_best_model(X,y):
    algos={
        'linear_regression': {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso':{
            'model': Lasso(),
            'params': {
                'alpha':[1,2],
                'selection': ['random','cyclic']
            }
        },
        'decision_tree':{
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['mse','friedman_mse'],
                'splitter' : ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name,config in algos.items():
        gs = GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model':algo_name,
            'best_score':gs.best_score_,
            'best_params':gs.best_params_
            })
        
    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model(X,y)


,model,best_score,best_params
0,linear_regression,0.292221,{'normalize': False}
1,lasso,-0.001446,"{'alpha': 1, 'selection': 'random'}"
2,decision_tree,0.772163,"{'criterion': 'friedman_mse', 'splitter': 'best'}"


From above we can see decision_tree model gives best score. lets check for logistic regression also as itis also used for classification 

In [169]:
from sklearn.linear_model import LogisticRegression

param=[{'C':[1,5,15,100,150,1000]}]
model2=LogisticRegression()
model=GridSearchCV(model2,param_grid=param,scoring='f1',cv=5)
model.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'C': [1, 5, 15, 100, 150, 1000]}], scoring='f1')

In [170]:
model.best_estimator_


LogisticRegression(C=1)

In [171]:
model.best_score_


0.783535727669595

In [172]:
model.best_params_

{'C': 1}

In [173]:
model.score(x_test,y_test)

0.7559943582510578

In [174]:
y_pred=model.predict(x_test)
y_pred
cm=confusion_matrix(y_test,y_pred)
y_pred=model.predict(x_test)
cm=confusion_matrix(y_test,y_pred)
score=accuracy_score(y_test,y_pred)
print(cm)
print(score)

[[1095  347]
 [ 345 1072]]
0.757957327736971


###### Support Vector Machine

In [175]:
from sklearn.svm import SVC
model = SVC()
model.fit(x_train,y_train)
model.score(x_test,y_test)
y_pred=model.predict(x_test)
cm=confusion_matrix(y_test,y_pred)
score=accuracy_score(y_test,y_pred)
print(cm)
print(score)

[[1139  303]
 [ 203 1214]]
0.8230150402238545


###### Naive Bayes

In [177]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train, y_train)
model.score(x_test,y_test)
y_pred=model.predict(x_test)
cm=confusion_matrix(y_test,y_pred)
score=accuracy_score(y_test,y_pred)
print(cm)
print(score)

ValueError: Negative values in data passed to MultinomialNB (input X)

###### Random Forest

In [178]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(x_train,y_train)
model.score(x_test,y_test)
y_pred=model.predict(x_test)
cm=confusion_matrix(y_test,y_pred)
score=accuracy_score(y_test,y_pred)
print(cm)
print(score)

[[1204  238]
 [ 126 1291]]
0.8726827562084645


###### From above we can see that Random Forest gives the best scole for given dataset